# Implementación el método de la máxima verosimilitud para la regresión logística

### Definir la función de entorno L(b)

In [1]:
from IPython.display import display, Math, Latex
display(Math(r'L(\beta)=\sum_{i=1}^n P_i^{y_i}(1-Pi)^{y_i}'))

<IPython.core.display.Math object>

In [40]:
def likelihood(y, pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1, len(y)+1))
    for i in range(len(y)):
        sum_in[i] = np.where(y[i]==1, pi[i], 1-pi[i])
        total_sum = total_sum * sum_in[i]
    return total_sum

### Calcular las probabilidades para cada observación

In [7]:
display(Math(r'P_i = P(x_i) = \frac{1}{1+e^{-\sum_{j=0}^k\beta_j\cdot x_{ij}}} '))

<IPython.core.display.Math object>

In [36]:
def logitprobs(X,beta):
    import numpy as np
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi=list(range(1,n_rows+1))
    expon=list(range(1,n_rows+1))
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex=X[i][j] * beta[j]
            expon[i] = ex + expon[i]
        with np.errstate(divide="ignore", invalid="ignore"):
            pi[i]=1/(1+np.exp(-expon[i]))
    return pi

### Calcular la matriz diagonal W

In [11]:
display(Math(r'W= diag(P_i \cdot (1-P_i))_{i=1}^n'))

<IPython.core.display.Math object>

In [12]:
def findW(pi):
    import numpy as np
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n)
    for i in range(n):
        print(i)
        W[i,i]=pi[i]*(1-pi[i])
        W[i,i].astype(float)
    return W

### Obtener la solución de la función logística

In [16]:
display(Math(r"\beta_{n+1} = \beta_n -\frac{f(\beta_n)}{f'(\beta_n)}"))
display(Math(r"f(\beta) = X(Y-P)"))
display(Math(r"f'(\beta) = XWX^T"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [42]:
def logistics(X, Y, limit):
    import numpy as np
    from numpy import linalg
    nrow = np.shape(X)[0]
    bias = np.ones(nrow).reshape(nrow,1)
    X_new = np.append(X, bias, axis = 1)
    ncol = np.shape(X_new)[1]
    beta = np.zeros(ncol).reshape(ncol,1)
    root_dif = np.array(range(1,ncol+1)).reshape(ncol,1)
    iter_i = 10000
    while(iter_i>limit):
        print("Iter:i"+str(iter_i) + ", limit:" + str(limit))
        pi = logitprobs(X_new, beta)
        print("Pi:"+str(pi))
        W = findW(pi)
        print("W:"+str(W))
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose())
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new))
        root_dif = np.array(linalg.inv(den)*num)
        beta = beta + root_dif
        print("Beta: "+str(beta))
        iter_i = np.sum(root_dif*root_dif)
        ll = likelihood(Y, pi)
    return beta

## Comprobación experimental

In [19]:
import numpy as np

In [20]:
X = np.array(range(10)).reshape(10,1)

In [21]:
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [22]:
Y = [0,0,0,0,1,0,1,0,1,1]

In [23]:
bias = np.ones(10).reshape(10,1)
X_new = np.append(X,bias,axis=1)

In [24]:
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [43]:
a = logistics(X,Y,0.00001)

Iter:i10000, limit:1e-05
Pi:[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
W:[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
Beta: [[ 0.43636364]
 [-2.36363636]]
Iter:i5.777190082644626, limit:1e-05
Pi:[array([0.08598797]), array([0.12705276]), array([0.18378532]), array([0.2583532]), array([0.35019508]), array([0.45467026]), array([0.56329497]), array([0.66616913]), array([0.75533524]), array([0.826

In [44]:
ll = likelihood(Y, logitprobs(X,a))

In [45]:
ll

array([1.32622426e-06])

In [ ]:
Y = 0.66220827 * X -3.69557172

# Con el paquete statsmodel de python

In [49]:
import statsmodels.api as sm

In [50]:
logit_model = sm.Logit(Y,X_new)

In [55]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [61]:
print(result.summary2())

                        Results: Logit
Model:              Logit            No. Iterations:   6.0000 
Dependent Variable: y                Pseudo R-squared: 0.360  
Date:               2018-04-05 18:54 AIC:              12.6202
No. Observations:   10               BIC:              13.2254
Df Model:           1                Log-Likelihood:   -4.3101
Df Residuals:       8                LL-Null:          -6.7301
Converged:          1.0000           Scale:            1.0000 
----------------------------------------------------------------
         Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
----------------------------------------------------------------
x1       0.6622     0.4001    1.6551   0.0979   -0.1220   1.4464
const   -3.6956     2.2889   -1.6145   0.1064   -8.1818   0.7906

